**This notebook is an exercise in the [SQL](https://www.kaggle.com/learn/intro-to-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/group-by-having-count).**

---


# Introduction
GROUP BY 是后面跟着要提取的列，这个列对应的数值会经过AVG COUNT这种类似的函数计算，比如国家——GDP的平均，动物——动物的数量，同样的分组指标会合并到一起

Queries with **GROUP BY** can be powerful. There are many small things that can trip you up (like the order of the clauses), but it will start to feel natural once you've done it a few times. Here, you'll write queries using **GROUP BY** to answer questions from the Hacker News dataset.

Before you get started, run the following cell to set everything up:

In [2]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex3 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


The code cell below fetches the `full` table from the `hacker_news` dataset.  We also preview the first five rows of the table.

In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("hacker_news", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "full" table
table_ref = dataset_ref.table("full")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()



Using Kaggle's public dataset BigQuery integration.


,title,url,text,dead,by,score,time,timestamp,type,id,parent,descendants,ranking,deleted
0,None,None,None,None,None,NaN,1438439367,2015-08-01 14:29:27+00:00,story,9987132,NaN,NaN,NaN,None
1,None,None,None,None,None,NaN,1438440244,2015-08-01 14:44:04+00:00,story,9987165,NaN,NaN,NaN,None
2,None,None,None,None,None,NaN,1438444828,2015-08-01 16:00:28+00:00,story,9987504,NaN,NaN,NaN,None
3,None,None,None,None,None,NaN,1438448896,2015-08-01 17:08:16+00:00,story,9987752,NaN,NaN,NaN,None
4,None,None,None,None,None,NaN,1260973504,2009-12-16 14:25:04+00:00,story,998789,NaN,NaN,NaN,None


In [ ]:
#查询一下

# Exercises

### 1) Prolific commenters

Hacker News would like to send awards to everyone who has written more than 10,000 posts. Write a query that returns all authors with more than 10,000 posts as well as their post counts. Call the column with post counts `NumPosts`.

In case sample query is helpful, here is a query you saw in the tutorial to answer a similar question:
```
query = """
        SELECT parent, COUNT(1) AS NumPosts
        FROM `bigquery-public-data.hacker_news.full`
        GROUP BY parent
        HAVING COUNT(1) > 10
        """
```

In [4]:
# Query to select prolific commenters and post counts
prolific_commenters_query = """
                                SELECT `by` AS author, COUNT(1) AS NumPosts
                                FROM `bigquery-public-data.hacker_news.full`
                                GROUP BY author
                                HAVING COUNT(1)>10000
                            """ # Your code goes here

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(prolific_commenters_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
prolific_commenters = query_job.to_dataframe()

# View top few rows of results
print(prolific_commenters.head())

# Check your answer
q_1.check()

       author  NumPosts
0         eru     23711
1  tokenadult     11001
2      kragen     15384
3    brudgers     21042
4       pjc50     25303


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [5]:
DeletedComments_query = """
                        SELECT COUNT(1) AS NumPosts 
                        FROM `bigquery-public-data.hacker_news.full`
                        WHERE `by` = 'eru'
                        """
# count(1) 直接计数
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(DeletedComments_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
deleted_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
deleted_comments.head()

,NumPosts
0,23711


For the solution, uncomment the line below.

In [4]:
q_1.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

prolific_commenters_query = """
                            SELECT `by` AS author, COUNT(1) AS NumPosts
                            FROM `bigquery-public-data.hacker_news.full`
                            GROUP BY author
                            HAVING COUNT(1) > 10000
                            """

```

### 2) Deleted comments

How many comments have been deleted? (If a comment was deleted, the `deleted` column in the table will have the value `True`.)

In [18]:
# Write your query here and figure out the answer
DeletedComments_query = """
                        SELECT deleted, COUNT(id)
                        FROM `bigquery-public-data.hacker_news.full`
                        GROUP BY deleted
                        """

# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(DeletedComments_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
deleted_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
deleted_comments.head()

,deleted,f0_
0,None,42115788


In [31]:
# Write your query here and figure out the answer
DeletedComments_query = """
                        SELECT COUNT(1) AS NumPosts 
                        FROM `bigquery-public-data.hacker_news.full`
                        WHERE deleted = True
                        """
# count(1) 直接计数
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(DeletedComments_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
deleted_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
deleted_comments.head()

,NumPosts
0,0


In [32]:
# Write your query here and figure out the answer
DeletedComments_query = """
                        SELECT COUNT(1) AS NumPosts 
                        FROM `bigquery-public-data.hacker_news.full`
                        
                        """
# count(1) 直接计数 不加where限定列的条件，计算所有行数
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(DeletedComments_query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
deleted_comments = query_job.to_dataframe()

# Print the first five rows of the DataFrame
deleted_comments.head()

,NumPosts
0,42115788


In [20]:
num_deleted_posts = 0 # Put your answer here

# Check your answer
q_2.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

For the solution, uncomment the line below.

In [19]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

# Query to determine how many posts were deleted
deleted_posts_query = """
                      SELECT COUNT(1) AS num_deleted_posts
                      FROM `bigquery-public-data.hacker_news.full`
                      WHERE deleted = True
                      """
                      
# Set up the query
query_job = client.query(deleted_posts_query)

# API request - run the query, and return a pandas DataFrame
deleted_posts = query_job.to_dataframe()

# View results
print(deleted_posts)

num_deleted_posts = 0

```

# Keep Going
**[Click here](https://www.kaggle.com/dansbecker/order-by)** to move on and learn about the **ORDER BY** clause.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-sql/discussion) to chat with other learners.*